# ML Pipelines
Now that we have access to the data, we need to get it into a form ready for ML training or inference.



In [2]:
import edit.data
import edit.pipeline

/g/data/eu56/hc7988/Projects/EDIT/data/src/edit/data/__init__.py:132: UserWarning: `edit` is under heavy development and may not continue to be supported.
  __python_warnings.warn("`edit` is under heavy development and may not continue to be supported.", UserWarning)


In [3]:
import yaml

We shall use the full config utilised for running FourCastNetV2

In [12]:
pipeline_config = """
edit.data.Catalog:
    ERA5:
      data_index: edit_archive_NCI.ERA5.ERA5
      name: ERA5
      args: []
      kwargs:
        level: single
        level_value: null
        resolution: reanalysis
        transforms: {}
        variables:
        - 10u
        - 10v
        - 2t
        - sp
        - msl
        - tcwv
    ERA5_0:
      data_index: edit_archive_NCI.ERA5.ERA5
      name: ERA5_0
      args: []
      kwargs:
        level: pressure
        level_value:
        - 50
        - 100
        - 150
        - 200
        - 250
        - 300
        - 400
        - 500
        - 600
        - 700
        - 850
        - 925
        - 1000
        resolution: reanalysis
        transforms: {}
        variables:
        - z
        - t
        - u
        - v
        - r
edit.pipeline.indexes.merge.MergeIndex:
  collapse_time: false
edit.pipeline.operations.reshape.DimensionFlatten:
  dimensions: level
edit.pipeline.operations.select.DropDataset:
  apply: false
  undo: true
  variables:
  - 100u
  - 100v
edit.pipeline.operations.transforms.operation.TransformOperation[long_force]:
  apply: true
  transforms:
    edit.data.transform.coordinates.standard_longitude:
      type: 0-360
  undo: false
edit.pipeline.operations.transforms.operation.TransformOperation[long_undo]:
  apply: false
  transforms:
    edit.data.transform.coordinates.standard_longitude:
      type: -180-180
  undo: true
edit.pipeline.operations.to_numpy.ToNumpy: {}
edit.pipeline.operations.reshape.Squish:
  axis: 1
edit.pipeline.operations.value.FillNan: {}
"""

In [13]:
pipeline = edit.pipeline.from_dict(yaml.load(pipeline_config, yaml.Loader))
pipeline

Data Pipeline
	Catalog                        Catalog of Data
	MergeIndex                     Merge Multiple DataIndexes together
	DimensionFlatten               Flatten and Expand on a Dimension
	DropDataset                    Drop Data Variables
	TransformOperation             Apply Transforms
	TransformOperation             Apply Transforms
	ToNumpy                        Convert to numpy arrays
	Squish                         Squish One Dimensional axis 1
	FillNan                        Fill nan's with 0 on both iteration and get

Now that we have this sequence of operations to be applied to the data when being loaded, we can index it just like we would a data source.

We shall index from step 5, so that the data is not converted to a numpy array.

In [18]:
pipeline.step(5)['2020-01-01T00']

<xarray.Dataset>
Dimensions:    (latitude: 721, time: 1, longitude: 1440)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2020-01-01
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables: (12/71)
    msl        (time, latitude, longitude) float32 dask.array<chunksize=(1, 182, 360), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(1, 182, 360), meta=np.ndarray>
    2t         (time, latitude, longitude) float32 dask.array<chunksize=(1, 182, 360), meta=np.ndarray>
    tcwv       (time, latitude, longitude) float32 dask.array<chunksize=(1, 182, 360), meta=np.ndarray>
    10u        (time, latitude, longitude) float32 dask.array<chunksize=(1, 182, 360), meta=np.ndarray>
    10v        (time, latitude, longitude) float32 dask.array<chunksize=(1, 182, 360), meta=np.ndarray>
    ...         ...
    z500       (time, latitude, longitude) float32 dask.array<chunksize=(1, 486, 360), meta=np.ndarray>
    z600       (time, latitude, longitude) float32 dask.array<chunksize=(1, 486, 360), meta=np.ndarray>
    z700       (time, latitude, longitude) float32 dask.array<chunksize=(1, 486, 360), meta=np.ndarray>
    z850       (time, latitude, longitude) float32 dask.array<chunksize=(1, 486, 360), meta=np.ndarray>
    z925       (time, latitude, longitude) float32 dask.array<chunksize=(1, 486, 360), meta=np.ndarray>
    z1000      (time, latitude, longitude) float32 dask.array<chunksize=(1, 486, 360), meta=np.ndarray>
Attributes:
    level-dtype:  int32